# Python Code for Quantum Artificial Bee Colony Spanning Tree Algorithm

**Sebelum menjalankan program pada section ini, jalankan dulu section berjudul "Python Class" di bawah section ini.**

* Jika program masuk ke infinite looping, berarti pada saat t=1 (looping pertama) program mengenerate suatu foodsource yang yang kebetulan jika dilakukan pengacakan sesuai prosedur pada paper (yakni mengambil dua "kotak" berisi 1 dan 0 kemudian menukarnya) tidak dapat diperoleh foodsource yang baru yang feasible walaupun dilakukan berulang kali. Hal ini dapat dibuktikan dengan melakukan tracing code pada program ini, yaitu dengan print "TRUE" jika diperoleh acakan yang feasible dan "FALSE" jika diperoleh acakan yang infeasible. Jika program terjebak di kasus ini, nantinya program akan terus menerus mencetak "FALSE" sampai pengguna melakukan interrupt pada Python.

* Jika program mengeluarkan ouput yang kira-kira berbunyi "fitvalue is not defined", berarti saat t=1 dan i=1 di Fase Employed Bee, program gagal memperoleh foodsource yang feasible walaupun telah dilakukan pengacakan sebanyak batas toleransinya (di sini dibuat 30). Hal ini bisa diakali dengan membuat kasus tambahan pada conditional statementnya, yaitu "jika t=1 dan i=1, acak terus foodsourcenya sampai dapat yang feasible". Hal ini sudah dilakukan, namun program pada akhirnya selalu terjebak di kasus pertama, yaitu infinite looping.

Jadi, jika terjadi infinite looping, pengguna dapat menginterrupt kernel dan menjalankan kembali program. Jika program mengeluarkan output "fitvalue is not defined", jalankan ulang program secara terus-menerus hingga output berupa tabel dan spanning tree minimum muncul.

In [17]:
import random
import numpy as np
from tabulate import tabulate
np.warnings.filterwarnings('ignore') #utk ngilangin warning karena adanya proses melakukan comparison dengan objek bukan number (NaN) berupa np.inf

#gerap={
#    'a':['b','c','e'],
#    'b':['a','d','e'],
#    'c':['d','e','a'],
#    'd':['c','b'],
#    'e':['c','a','b']
#}

graft = { "v1"  : ["v2","v3"],
          "v2"  : ["v1","v3","v5"],
          "v3"  : ["v1","v2","v4","v5","v6"],
          "v4"  : ["v3","v6"],
          "v5"  : ["v2","v3","v6","v7","v8"],
          "v6"  : ["v3","v4","v5","v8","v9"],
          "v7"  : ["v5","v8","v10"],
          "v8"  : ["v5","v6","v7","v9","v10","v11","v13"],
          "v9"  : ["v6","v8","v11","v12"],
          "v10" : ["v7","v8","v13","v14"],
          "v11" : ["v8","v9","v12","v13"],
          "v12" : ["v9","v11","v13","v15"],
          "v13" : ["v8","v10","v11","v12","v14","v15"],
          "v14" : ["v10","v13","v15","v16"],
          "v15" : ["v12","v13","v14","v16"],
          "v16" : ["v14","v15"]
        }


graf=Graph(graft)
edges=graf.edges()
E=len(edges)
nodes=graf.vertices()
N=len(nodes)
weight = [74.96,106.65,121.91,89.01,
          102.04,168.74,74.58,113.1,
          124.37,87.46,117.44,87.34,
          128.7,121.23,93.69,132.65,
          152.89,67.09,112.92,97.3,
          128.11,115.62,88.92,115.05,
          80.28,129.02,88.9,109.35,
          123.87,166.69,103.51,84.69]


population=20
loop=50


def GenerateGrafdariAcakan(subgraf): #func utk menyatakan bil biner 1101... yg diacak di encoding mech spy jadi graf
    h = {'v1': [],
    'v2': [],
    'v3': [],
    'v4': [],
    'v5': [],
    'v6': [],
    'v7': [],
    'v8': [],
    'v9': [],
    'v10': [],
    'v11': [],
    'v12': [],
    'v13': [],
    'v14': [],
    'v15': [],
    'v16': []}
    edgeyangmuncul = []
    for l in range(0,len(subgraf)):
        if subgraf[l]==1:
            edgeyangmuncul.append(list(edges[l]))
    #Kita bentuk grafnya dari graf kosongan dengan menambah edge2 dari variabel edgeyangmuncul
    for l in edgeyangmuncul:
        h[l[0]].append(l[1])
        h[l[1]].append(l[0])
    
    grafnew=Graph(h)
    
    connect_ga = grafnew.is_connected()
    return grafnew,connect_ga

def GenFoodSource(E, N, population): #generate food source. Food source sama tdk mslh, nanti diacak lagi sama employed
    foodsource = []
    fitness = []
    
    #Acak list subgraf dan generate acakannya sbyk population
    for i in range(0,population):
        #construct list dengan elemen sbyk E yang isinya "1" sbyk N-1
        subgraf = [1]*(N-1)
        for j in range(0,E-(N-1)):
            subgraf.append(0)
        random.shuffle(subgraf) #acak list subgraf
        foodsource.append(subgraf) #masukin sbg elemen foodsource
        #Construct graf yang bersesuaian dgn acakan kita
        grafnew,connect_ga=GenerateGrafdariAcakan(subgraf)
        #Terhubung ga? Kalo terhubung -> spanning tree -> feasible. Terus hitung fitness value
        if connect_ga==True:
            fitvalue=0
            for k in range(0,len(subgraf)):
                fitvalue+=subgraf[k]*weight[k]
            fitness.append(fitvalue)
        else:
            fitness.append(np.inf)
    return foodsource,fitness



#####---INISIALISASI FOOD SOURCE---#####
foodsource,fitness = GenFoodSource(E,N,population)
#print('foodsource hasil acak\n',foodsource)
#print('fitness hasil acak\n',fitness)

for t in range(1,loop+1):
    #####---FASE EMPLOYED BEE---#####
    count = [0]*population #utk ngitung udh berapa kali suatu foodsource ga diupadte. Kalo ga diudpate stlh sekian kali-> discard, generate a new one
    for i in range(0,population):
        kondisi=0
        #Replacement Strategy no 1
        if count[i]<=30:
            while kondisi==0:
                foodsource_baru=[]
                for f in foodsource[i]:
                    foodsource_baru.append(f)
                cek=0
                #mulai pilih dua posisi random, tapi pastiin di kedua posisi random ini komponennya berbeda
                while cek==0:
                    p=random.randint(0,E-1)
                    if foodsource_baru[p]==1: #komponen ke-p dari foodsource haruslah 1
                        cek=1
                cek=0
                while cek==0:
                    q=random.randint(0,E-1)
                    if foodsource_baru[q]!=foodsource_baru[p]: #kalo komponen ke-q beda dari komp ke-p baru boleh exit dari loop
                        cek=1
                #tuker posisi
                foodsource_baru[p],foodsource_baru[q]=foodsource_baru[q],foodsource_baru[p]
                if foodsource_baru not in foodsource:
                    grafnew,connect_ga=GenerateGrafdariAcakan(foodsource_baru)
                    if connect_ga==True:
                        fitvalue=0
                        for k in range(0,len(foodsource_baru)):
                            fitvalue+=foodsource_baru[k]*weight[k]
                        kondisi=1
                    if kondisi==0:
                        count[i]+=1
                    if count[i]>30:
                        kondisi = 1
            #Replacement Strategy no2
            if fitness.count(np.inf)>0 and count[i]<=30:
                cek=0
                while cek==0:
                    r=random.randint(0,len(fitness)-1)
                    if fitness[r]==np.inf:
                        foodsource[r]=foodsource_baru
                        count[i],count[r]=0,0
                        fitness[r]=fitvalue
                        cek=1
            #Replacement Strategy no3
            elif fitvalue < fitness[i] and count[i]<=30:
                foodsource[i]=foodsource_baru
                count[i]=0
                fitness[i]=fitvalue
            else:
                count[i]+=1

    #print('\nfoodsource hasil employed\n',foodsource)
    #print('fitness hasil emploed\n',fitness)



    #####---ONLOOKER BEE---#####
    fitnesssum = sum(fitness) #sum semua nilai fitness. Di fase ini harusnya kan semua foodsource udh feasible gegara proses employed bee
    probability = []
    for i in fitness:
        probability.append(i/fitnesssum) #generate list isinya probability tiap food source terpilih
    urutanfsterpilih = int(np.random.choice(len(foodsource),1,p=probability)) #ambil satu food source acak dgn probabilitas yg udah ditentuin
    fs_chosen = foodsource[urutanfsterpilih]
    #print('foodsource ke berapa yg terpilih?:',urutanfsterpilih)

    #Ngurutin Foodsource dan Fitnessnya
    def bubbleSort(fitness,foodsource):
        n = len(fitness)
        arr=[]
        for f in fitness:
            arr.append(f)
        foodsource1=[]
        for f in foodsource:
            foodsource1.append(f)
        # Traverse through all array elements 
        for i in range(n):

            # Last i elements are already in place 
            for j in range(0, n-i-1):

                # traverse the array from 0 to n-i-1 
                # Swap if the element found is greater 
                # than the next element 
                if arr[j] > arr[j+1]: 
                    arr[j], arr[j+1] = arr[j+1], arr[j]
                    foodsource1[j], foodsource1[j+1] = foodsource1[j+1], foodsource1[j] #tuker posisi foodsource juga
                    #fitness[j],fitness[j+1]=fitness[j+1],fitness[j] #tuker psisi fitness
        return foodsource1


    sortedfs = bubbleSort(fitness,foodsource) #generate list foodsource yg terurut
    j=random.randint(0,E-1)
    nmb=0
    for f in range(0,int(len(sortedfs)/2)):
        nmb+=sortedfs[f][j]
    nmb=nmb/int(len(sortedfs)/2)
    a=random.uniform(0,1)
    pe=a*fs_chosen[j]+(1-a)*sortedfs[0][j]
    b = 1-t/(2*loop)
    u=random.uniform(0,1)
    if u>=0.5:
        q_j=pe-b*abs(nmb-fs_chosen[j])*np.log(1/u)
    else:
        q_j=pe+b*abs(nmb-fs_chosen[j])*np.log(1/u)
    if q_j>=0.5:
        p_j=1
    else:
        p_j=0
    #print('apakah p_j dan komponen ke-j dari fs terpilihnya sama?:',p_j==fs_chosen[j])
    if p_j != fs_chosen[j]:
        cek=0
        while cek==0 and count[urutanfsterpilih]<=30:
            ka=random.randint(0,E-1)
            if fs_chosen[ka]==p_j:
                temp=fs_chosen[ka]
                fs_chosen[ka]=fs_chosen[j]
                fs_chosen[j]=temp
                #print('hasil tuker: ',fs_chosen)
                chosen_graf,connect_ga=GenerateGrafdariAcakan(fs_chosen)
                if connect_ga==True:
                    cek=1
                    fitvalue=0
                    for i in range(0,len(fs_chosen)):
                        fitvalue+=fs_chosen[i]*weight[i]
                    #print(fitvalue)
                    if fitvalue < fitness[urutanfsterpilih] and fs_chosen not in foodsource:
                        foodsource[urutanfsterpilih]=fs_chosen
                        fitness[urutanfsterpilih]=fitvalue
                        count[urutanfsterpilih]=0
                    else:
                        temp=fs_chosen[ka]
                        fs_chosen[ka]=fs_chosen[j]
                        fs_chosen[j]=temp
                        count[urutanfsterpilih]+=1
                else:
                    temp=fs_chosen[ka]
                    fs_chosen[ka]=fs_chosen[j]
                    fs_chosen[j]=temp
    
    #####---ScoutBee---#####
    
    def GenerateANewOne():
        subgraf = [1]*(N-1)
        for j in range(0,E-(N-1)):
            subgraf.append(0)
        random.shuffle(subgraf)
        fitvalue=0
        for k in range(0,len(subgraf)):
            fitvalue+=subgraf[k]*weight[k]
        return subgraf,fitvalue
    
    for i in range(0,len(count)):
        if count[i]>30:
            cek=0
            while cek==0:
                gen_fs_baru,fit_baru=GenerateANewOne()
                if gen_fs_baru not in foodsource:
                    foodsource[i],fitness[i]=gen_fs_baru,fit_baru
                    count[i]=0
                    cek=1
    



''' 
#Cara lain menampilkan semua ST, tapi ga rapi
print('***Daftar Semua Spanning Tree***')
for i in range(len(foodsource)):
    edgemuncul=[]
    for z in range(0,E):
        if foodsource[i][z]==1:
            edgemuncul.append(z+1)
    print('Edge yang muncul di MST (nomor edgenya, ada di paper):\n',edgemuncul)
    print('Weight value dari spanning tree tsb:\n',fitness[i])
'''

tabel=[]
for i in range(0,population):
    tambahin=[]
    tambahin.append(i+1)
    tambahin.append(fitness[i])
    edgemuncul=[]
    for z in range(0,E):
        if foodsource[i][z]==1:
            edgemuncul.append(z+1)
    tambahin.append(edgemuncul)
    tabel.append(tambahin)
print('Spanning Tree Found by QABCST')
print(tabulate(tabel,headers=['No.','Weight','Edge Series'],tablefmt='orgtbl'))
print()

print('Spanning Tree paling minimum diberikan oleh:')
stmin=min(fitness)
for i in range(0,len(fitness)):
    MST=[]
    if stmin==fitness[i]:
        MST.append(foodsource[i])
        indexmst=0
        edgediMST=[]
        for z in range(0,E):
            if MST[indexmst][z]==1:
                edgediMST.append(z+1)
        indexmst+=1
        print('Edge Series:',edgediMST)
        print('Weight:',fitness[i])

Spanning Tree Found by QABCST
|   No. |   Weight | Edge Series                                              |
|-------+----------+----------------------------------------------------------|
|     1 |  1337.48 | [1, 2, 4, 7, 8, 10, 12, 15, 18, 20, 23, 25, 27, 31, 32]  |
|     2 |  1357.99 | [1, 4, 7, 8, 10, 12, 15, 18, 20, 22, 23, 24, 25, 27, 32] |
|     3 |  1340.18 | [1, 4, 7, 8, 10, 12, 15, 18, 20, 23, 25, 27, 28, 31, 32] |
|     4 |  1416.86 | [1, 2, 4, 7, 8, 10, 15, 19, 20, 23, 24, 25, 27, 28, 32]  |
|     5 |  1401.93 | [1, 2, 7, 8, 10, 11, 12, 18, 20, 23, 25, 27, 28, 29, 32] |
|     6 |  1418.62 | [1, 4, 7, 8, 10, 13, 15, 18, 21, 22, 23, 25, 27, 31, 32] |
|     7 |  1359.93 | [1, 4, 5, 7, 10, 12, 15, 18, 21, 23, 25, 27, 28, 31, 32] |
|     8 |  1368.77 | [1, 2, 4, 5, 7, 10, 12, 15, 18, 21, 23, 24, 25, 27, 32]  |
|     9 |  1374.72 | [1, 2, 4, 5, 7, 10, 12, 18, 19, 20, 23, 27, 28, 31, 32]  |
|    10 |  1420.04 | [1, 2, 5, 7, 9, 11, 15, 18, 20, 22, 23, 25, 27, 31, 32]  |
|    11 | 

# Python Class: Graph (Run This First)

In [1]:
class Graph(object):

    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict

    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self.__graph_dict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self.__graph_dict:
            self.__graph_dict[vertex] = []

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list; 
            between two vertices can be multiple edges! 
        """
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.__graph_dict:
            self.__graph_dict[vertex1].append(vertex2)
        else:
            self.__graph_dict[vertex1] = [vertex2]

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            for neighbour in self.__graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges

    def __str__(self):
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def find_isolated_vertices(self):
        """ returns a list of isolated vertices. """
        graph = self.__graph_dict
        isolated = []
        for vertex in graph:
            print(isolated, vertex)
            if not graph[vertex]:
                isolated += [vertex]
        return isolated
    
    def is_connected(self, 
                     vertices_encountered = None, 
                     start_vertex=None):
        """ determines if the graph is connected """
        if vertices_encountered is None:
            vertices_encountered = set()
        gdict = self.__graph_dict        
        vertices = list(gdict.keys()) # "list" necessary in Python 3 
        if not start_vertex:
            # chosse a vertex from graph as a starting point
            start_vertex = vertices[0]
        vertices_encountered.add(start_vertex)
        if len(vertices_encountered) != len(vertices):
            for vertex in gdict[start_vertex]:
                if vertex not in vertices_encountered:
                    if self.is_connected(vertices_encountered, vertex):
                        return True
        else:
            return True
        return False